# Data Plotter

This jupyter notebook plots average runtime, success rate, and flowtime ratio from the data stored in the "Data" directory generated by the Data Extractor notebook. 

Plots are automatically generated and stored in a "Figures" directory. All directories are automatically created if they do not already exist. Within the "Figures" directory, grouped map plots and individual map plots are separated.

## Dependencies and Initializations

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import os
import csv

In [ ]:
algorithms = ["CBS", "PBS"]
resolutions = [1, 2, 4]

map_names = ["empty-8-8", "empty-16-16", "empty-32-32", "empty-48-48", 
             "random-32-32-10", "random-32-32-20", "random-64-64-10", "random-64-64-20", 
             "room-32-32-4", "room-64-64-8",  "room-64-64-16",
             "maze-32-32-2", "maze-32-32-4", "maze-128-128-10", "maze-128-128-2", 
             "Berlin_1_256", "Boston_0_256", "Paris_1_256", 
             "ht_chantry", "ht_mansion_n", "lak303d", "lt_gallowstemplar_n", "den312d", "ost003d", 
             "brc202d", "den520d", "w_woundedcoast"]

A Directory to save figures is created, if it does not exist. This directory separates map group figures from individual map figures. 

In [ ]:
fig_dir = os.path.join(os.getcwd(), "Figures")
group_dir = os.path.join(fig_dir, "GroupedFigures")
ind_dir = os.path.join(fig_dir, "IndividualFigures")

if not os.path.isdir(fig_dir): 
    os.mkdir(fig_dir)
    print("Making a new directory to hold all figures.")
    
if not os.path.isdir(group_dir): 
    os.mkdir(group_dir)
    print("Making a new directory to hold all grouped figures.")
    
if not os.path.isdir(ind_dir): 
    os.mkdir(ind_dir)
    print("Making a new directory to hold all individual figures.")

## Load CSV Data

This will load in all of the statistics exported from the ExportData notebook. These file paths can be modified as necessary. 

In [ ]:
data_dir = os.path.join(os.getcwd(), "Data")
time_file = os.path.join(data_dir, "time_stats.csv")
cost_file = os.path.join(data_dir, "cost_ratios.csv")
count_file = os.path.join(data_dir, "solved_count.csv")
success_file = os.path.join(data_dir, "success_rate.csv")

#### Get Time Related Statistics: 

In [ ]:
time_stats = dict()

# Initialize map using known keys
for algorithm in algorithms: 
    time_stats[algorithm] = dict()
    for res in resolutions: 
        time_stats[algorithm][res] = dict()
        for name in map_names: 
            time_stats[algorithm][res][name] = dict()
            time_stats[algorithm][res][name]["total_time"] = dict()
            time_stats[algorithm][res][name]["num_solved_scen"] = dict()
            time_stats[algorithm][res][name]["average_time"] = dict()
            

# Parse data file and store values in map
with open (time_file, 'r', newline='') as csvfile: 
    csv_reader = csv.reader(csvfile, delimiter=",")
    next(csv_reader)
    
    for row in csv_reader: 
        algorithm, resolution, name, agents, total, solved, average = row
        resolution, agents, solved = int(resolution), int(agents), int(solved)
        total, average = float(total), float(average)
        
        time_stats[algorithm][resolution][name]["total_time"][agents] = total
        time_stats[algorithm][resolution][name]["num_solved_scen"][agents] = solved
        time_stats[algorithm][resolution][name]["average_time"][agents] = average

#### Get Cost Related Statistics: 

In [ ]:
cost_stats = dict()

# Initialize map using known keys
for res in resolutions: 
    cost_stats[res] = dict()
    for name in map_names: 
        cost_stats[res][name] = dict()

# Parse data file and store values in map, new keys added as needed
with open(cost_file, "r", newline='') as csvfile: 
    csv_reader = csv.reader(csvfile, delimiter=",")
    next(csv_reader)
    
    for row in csv_reader: 
        res, name, scen, agents, ratio = row
        res, scen, agents = int(res), int(scen), int(agents)
        ratio = float(ratio)
        
        if agents not in cost_stats[res][name]: 
            cost_stats[res][name][agents] = dict()
        if scen not in cost_stats[res][name][agents]: 
            cost_stats[res][name][agents][scen] = dict()
            
        cost_stats[res][name][agents][scen] = ratio        

#### Get Maximum and Minimum Problem Size Statistics: 

In [ ]:
count_stats = dict()

# Initialize map with known keys
for algorithm in algorithms:
    count_stats[algorithm] = dict()
    for res in resolutions:
        count_stats[algorithm][res] = dict()
        for name in map_names:
            count_stats[algorithm][res][name] = dict()

# Parse data file and store values in map
with open(count_file, "r", newline='') as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=",")
    next(csv_reader)

    for row in csv_reader:
        algorithm, resolution, name, min_num, max_num = row
        resolution, min_num, max_num = int(resolution), int(min_num), int(max_num)

        count_stats[algorithm][resolution][name]["min_agents"] = min_num
        count_stats[algorithm][resolution][name]["max_agents"] = max_num

#### Get Success Rate Statistics: 

In [ ]:
success_stats = dict()

# Initialize map with known keys
for algorithm in algorithms:
    success_stats[algorithm] = dict()
    for res in resolutions:
        success_stats[algorithm][res] = dict()
        for name in map_names:
            success_stats[algorithm][res][name] = dict()

# Parse data file and store values in map           
with open (success_file, "r", newline='') as csvfile: 
    csv_reader = csv.reader(csvfile, delimiter=",")
    next(csv_reader)
    
    for row in csv_reader: 
        algorithm, resolution, name, num_agents, rate = row
        resolution, num_agents = int(resolution), int(num_agents)
        rate = float(rate)
        
        success_stats[algorithm][resolution][name][num_agents] = rate

## Generate Average Plots

Averaged plots are determined using map groupings. CBS and PBS are colored using the colors below. These colors go from light to dark and are used for the different map resolution results. 

In [ ]:
cbs_color = ['#ffb3a7', '#eb6574', '#93003a'] # warm colors (light to dark)
pbs_color = ['#9dced6', '#6694c1', '#00429d'] # cool colors (light to dark)

cbs_label = ['CBS, Res1', 'CBS, Res2', 'CBS, Res4']
pbs_label = ['PBS, Res1', 'PBS, Res2', 'PBS, Res4']

In [ ]:
map_groupings = {"Empty": ["empty-8-8", "empty-16-16", "empty-32-32", "empty-48-48"],
                 "Random": ["random-32-32-10", "random-32-32-20", "random-64-64-10", "random-64-64-20"],
                 "Rooms":  ["room-32-32-4", "room-64-64-8", "room-64-64-16"],
                 "Maze": ["maze-32-32-2", "maze-32-32-4", "maze-128-128-2", "maze-128-128-10"], 
                 "Cities": ["Berlin_1_256", "Boston_0_256", "Paris_1_256"],
                 "Games-Small": ["ht_chantry", "ht_mansion_n", "lak303d", "lt_gallowstemplar_n", "den312d", "ost003d"],
                 "Games-Large": ["brc202d", "den520d", "w_woundedcoast"]}               

#### Plot Group Runtime Average

In [ ]:
def GetGroupRuntimeAverage(algorithm, res, group): 
    # get maximum problem size
    all_keys = set()
    for name in group: 
        keys = time_stats[algorithm][res][name]["total_time"].keys()
        all_keys.update(keys)
    max_agents = max(all_keys)
    
    # initialize data structures holding summed values and number of solved scenarios
    summed_values = [0 for _ in range(0, max_agents + 1, 2)]
    num_groups = [0 for _ in range(0, max_agents + 1, 2)]
    
    # extract maps for information and populate lists
    for name in group: 
        for num_agents in time_stats[algorithm][res][name]["total_time"].keys(): 
            value = time_stats[algorithm][res][name]["total_time"][num_agents]
            count = time_stats[algorithm][res][name]["num_solved_scen"][num_agents]
            
            summed_values[int(num_agents/2 - 1)] += value
            num_groups[int(num_agents/2 - 1)] += count
 
    # average retrieved times based on the total number of solved scenarios
    # values are returned as an (agent_num, average_time) tuple pair
    averaged_values = []
    for i in range(len(summed_values)): 
        if num_groups[i] == 0: 
            break
        averaged_values.append(((i + 1) * 2, summed_values[i] / num_groups[i]))
        
    return averaged_values

In [ ]:
for group_name, group in map_groupings.items():
    # force figure size to (8, 5)
    plt.figure(figsize=(8, 5))
    
    # collect group averaged data
    cbs_data = []
    pbs_data = []
    for res in resolutions: 
        cbs_data.append(GetGroupRuntimeAverage("CBS", res, group))
        pbs_data.append(GetGroupRuntimeAverage("PBS", res, group))
    
    # find maximum problem size to customize x-axis limit
    max_x = 0
    
    # Plot CBS data
    for i in range(len(cbs_data)): 
        x, y = zip(*cbs_data[i])
        line, = plt.plot(x, y, marker='s', linestyle='-', markersize=4, linewidth=2, color=cbs_color[i], label=cbs_label[i])
        max_x = max(max_x, max(x))
        
    # Plot PBS data
    for i in range(len(pbs_data)): 
        x, y = zip(*pbs_data[i])
        line, = plt.plot(x, y, marker='o', linestyle='-', markersize=4, linewidth=2, color=pbs_color[i], label=pbs_label[i])
        max_x = max(max_x, max(x))
    
    # Customize labels, titles, legends, etc., as needed
    plt.xticks(range(0, max_x+3, 4))
    plt.xlabel('Agents')
    plt.ylabel('Runtime (s)')
    plt.title('Average Runtime of ' + group_name + " Group")
    plt.legend(loc='upper left', fontsize=9)  # Add legend if desired

    # Save and Show the plot
    plt.savefig("Figures/GroupedFigures/AverageTime_" + group_name + ".png")
    plt.show()

#### Plot Group Success Rate Averages

In [ ]:
def GetGroupSuccessAverage(algorithm, res, group): 
    
    # Get maximum problem size
    all_keys = set()
    for name in group: 
        keys = success_stats[algorithm][res][name].keys()
        all_keys.update(keys) 
    max_agents = max(all_keys)
    
    # initialize data structures holding summed values and number of solved scenarios 
    summed_values = [0 for _ in range(0, max_agents + 1, 2)]
    num_groups = [0 for _ in range(0, max_agents + 1, 2)]
    
    # extract maps for information and populate lists
    for name in group: 
        for num_agents in success_stats[algorithm][res][name].keys(): 
            value = success_stats[algorithm][res][name][num_agents]
            
            summed_values[int(num_agents/2 - 1)] += value
            num_groups[int(num_agents/2 - 1)] += 1
            
    # average retrieved success rates are  based on the total number maps in a group
    # values are returned as an (agent_num, average_time) tuple pair
    averaged_values = []
    for i in range(len(summed_values)): 
        if num_groups[i] == 0: 
            break
        average = summed_values[i] / num_groups[i]            
        averaged_values.append(((i + 1) * 2, average))
        
        # early break remove trailing zeros after the algorithm fails for all scenarios
        if average == 0: 
            break
        
    return averaged_values




In [ ]:
for group_name, group in map_groupings.items():
    # force figure size to (8, 5)
    plt.figure(figsize=(8, 5))
            
    # collect group averaged data
    cbs_data = []
    pbs_data = []
    for res in resolutions: 
        cbs_data.append(GetGroupSuccessAverage("CBS", res, group))
        pbs_data.append(GetGroupSuccessAverage("PBS", res, group))

    # find maximum problem size to customize x-axis limit
    max_x = 0
    
    # plot cbs data
    for i in range(len(cbs_data)): 
        x, y = zip(*cbs_data[i])
        line, = plt.plot(x, y, marker='s', linestyle='-', markersize=4, linewidth=2, color=cbs_color[i], label=cbs_label[i])
        max_x = max(max(x), max_x)
    
    # plot pbs data
    for i in range(len(pbs_data)): 
        x, y = zip(*pbs_data[i])
        line, = plt.plot(x, y, marker='o', linestyle='-', markersize=4, linewidth=2, color=pbs_color[i], label=pbs_label[i])
        max_x = max(max(x), max_x)
    
    # Customize labels, titles, legends, etc., as needed
    plt.xticks(range(0, max_x+3, 4))
    plt.xlabel('Agents')
    plt.ylabel('Success Rate')
    plt.title('Success Rate of ' + group_name + " Group")
    plt.legend(loc='upper right', fontsize=9)
    
    # Save and Show the plot
    plt.savefig("Figures/GroupedFigures/SuccessRate_" + group_name + ".png")
    plt.show()

#### Plot Group Flowtime Ratios

In [ ]:
# Flowtime Ratio

for group_name, group in map_groupings.items(): 
    
    # get maximum problem size
    max_agent = 0
    for res in resolutions: 
        for name in group: 
            for agent_num in cost_stats[res][name].keys(): 
                max_agent = max(max_agent, agent_num)    
                
    # initialize x-axis and empty structures for cbs and pbs data
    x_data = list(range(2, max_agent + 1, 2))
    cbs_data = [[[] for _ in x_data] for _ in resolutions]
    pbs_data = [[[] for _ in x_data] for _ in resolutions]

    # extract all cost ratios for cbs and pbs at the different resolutions
    all_values = []
    for i in range(len(resolutions)): 
        res = resolutions[i]
        for name in group: 
            for agent_num in cost_stats[res][name].keys(): 
                for scen in cost_stats[res][name][agent_num].keys():
                    cbs_data[i][int(agent_num/2 - 1)].append(1.0)
                    pbs_data[i][int(agent_num/2 - 1)].append(cost_stats[res][name][agent_num][scen])
                    
                    all_values.append(cost_stats[res][name][agent_num][scen])
    
    # create three subplots - one for each resolution
    fig, axs = plt.subplots(3, 1, sharex=True, figsize=(8,5))   
    
    # track maximum flowtime ratio 
    max_value = 0
    offset = 0.2
    for i in range(len(resolutions)): 
        for j, x in enumerate(x_data): 
            if len(pbs_data[i][j]) == 0: 
                continue
                
            # offset data for a given x-tick
            true_x = np.linspace(x-offset, x+offset, len(cbs_data[i][j]))
            axs[i].scatter(true_x, cbs_data[i][j], color=cbs_color[i], marker="s", s=20)
            axs[i].scatter(true_x, pbs_data[i][j], color=pbs_color[i], marker="o", s=6)
            
            max_value = max(max_value, max(pbs_data[i][j]))
    
    # round max_value to the nearest thousandth place or 1.003 if all values are 1.0
    max_value = round(max_value, 3)
    if max_value == 1.0: 
        max_value = 1.003
    y_off = (max_value - 1.00) / 10
    
    for i in range(len(resolutions)): 
        # add y-axis for each subplot
        axs[i].set_ylabel("Flowtime Ratio \nfor Res " + str(resolutions[i]), fontsize=9)
        
        # create dummy points to create small legends
        cbs_dummy = Line2D([0], [0], marker='s', color='w', markerfacecolor=cbs_color[i], markersize=7, label=cbs_label[i])
        pbs_dummy = Line2D([0], [0], marker='o', color='w', markerfacecolor=pbs_color[i], markersize=6, label=pbs_label[i])
        axs[i].legend(handles=[cbs_dummy, pbs_dummy], fontsize=8)
        
        # set y-axis limits 
        axs[i].set_ylim(1 - y_off, max_value)
        axs[i].tick_params(axis='y', labelsize=8)
        
    fig.suptitle('Suboptimality Comparison of ' + group_name + " Group",
                x=0.555, horizontalalignment='center')    
     
    # Customize labels, titles, legends, etc., as needed           
    plt.xlabel('Agents', fontsize=9)
    plt.xticks(x_data, fontsize = 8)    
    plt.tight_layout(rect=[0.05, 0.03, 1, 1])
    
    plt.savefig("Figures/GroupedFigures/CostRatios_" + group_name + ".png")
    plt.show()
    

## Generate All Plots

Generates plots for each individual map. Uses the same colorscheme as defined above. 

#### Plot Individual Runtime Averages

In [ ]:
def GetIndividualRuntimeAverage(algorithm, res, name):
    # extract runtime average points in increasing problem size order 
    keys = list(time_stats[algorithm][res][name]["average_time"].keys())
    keys.sort()
    
    values = []
    for key in keys: 
        values.append((key, time_stats[algorithm][res][name]["average_time"][key]))

    return values


In [ ]:
for name in map_names: 
    plt.figure(figsize=(8, 5))
    
    # Get all cbs and pbs data
    cbs_data = []
    pbs_data = []
    for res in resolutions: 
        cbs_data.append(GetIndividualRuntimeAverage("CBS", res, name))
        pbs_data.append(GetIndividualRuntimeAverage("PBS", res, name))
    
    # Plot CBS data
    max_x = 0
    for i in range(len(cbs_data)): 
        if len(cbs_data[i]) == 0: 
            continue
        x, y = zip(*cbs_data[i])
        line, = plt.plot(x, y, marker='s', linestyle='-', markersize=4, linewidth=2, color=cbs_color[i], label=cbs_label[i])
        max_x = max(max_x, max(x))
        
    # Plot PBS data
    for i in range(len(pbs_data)): 
        if len(pbs_data[i]) == 0: 
            continue
        x, y = zip(*pbs_data[i])
        line, = plt.plot(x, y, marker='o', linestyle='-', markersize=4, linewidth=2, color=pbs_color[i], label=pbs_label[i])
        max_x = max(max_x, max(x))
    
    # Customize labels, titles, legends, etc., as needed
    plt.xticks(range(0, max_x+3, 4))
    plt.ylim(0, 60)
    plt.xlabel('Agents')
    plt.ylabel('Runtime (s)')
    plt.title('Average Runtime of ' + name)
    plt.legend(loc='upper left', fontsize=9)  # Add legend if desired

    # Save and Show the plot
    plt.savefig("Figures/IndividualFigures/AverageTime_" + name + ".png")
    plt.show()

#### Plot Individual Success Rates

In [ ]:
def GetIndividualSuccessRate(algorithm, res, name): 
    # extract success rate points in increasing problem size order 
    keys = list(success_stats[algorithm][res][name].keys())
    keys.sort()
    
    values = []
    for key in keys: 
        value = success_stats[algorithm][res][name][key]
        values.append((key, value))
        
        if value == 0: 
            break
        
    return values


In [ ]:
for name in map_names: 
    plt.figure(figsize=(8, 5))
    
    # get all cbs and pbs data
    cbs_data = []
    pbs_data = []
    for res in resolutions: 
        cbs_data.append(GetIndividualSuccessRate("CBS", res, name))
        pbs_data.append(GetIndividualSuccessRate("PBS", res, name))

    max_x = 0
    
    # plot cbs data
    for i in range(len(cbs_data)):  
        if len(cbs_data[i]) == 0: 
            continue
        x, y = zip(*cbs_data[i])
        line, = plt.plot(x, y, marker='s', linestyle='-', markersize=4, linewidth=2, color=cbs_color[i], label=cbs_label[i])
        max_x = max(max_x, max(x))
        
    # plot pbs data
    for i in range(len(pbs_data)):  
        if len(pbs_data[i]) == 0: 
            continue
        x, y = zip(*pbs_data[i])
        line, = plt.plot(x, y, marker='o', linestyle='-', markersize=4, linewidth=2, color=pbs_color[i], label=pbs_label[i])
        max_x = max(max_x, max(x))
    
    # Customize labels, titles, legends, etc., as needed
    plt.xticks(range(0, max_x+3, 4))
    plt.xlabel('Agents')
    plt.ylabel('Success Rate')
    plt.title('Success Rate of ' + name)
    plt.legend(loc='upper left', fontsize=9)  # Add legend if desired

    # Save and Show the plot
    plt.savefig("Figures/IndividualFigures/SuccessRate_" + name + ".png")
    plt.show()

#### Plot Individual Flowtime Ratios

In [ ]:
for name in map_names: 
    
    # get maximum problem size
    max_agent = 0
    for res in resolutions: 
        for agent_num in cost_stats[res][name].keys(): 
            max_agent = max(max_agent, agent_num)

    # initialize x-axis and empty structures for cbs and pbs data 
    x_data = list(range(2, max_agent + 1, 2))
    cbs_data = [[[] for _ in x_data] for _ in resolutions]
    pbs_data = [[[] for _ in x_data] for _ in resolutions]

    # get all cbs and pbs flowtime ratio values
    all_values = []
    for i in range(len(resolutions)): 
        res = resolutions[i]
        for agent_num in cost_stats[res][name].keys(): 
            for scen in cost_stats[res][name][agent_num].keys():
                cbs_data[i][int(agent_num/2 - 1)].append(1.0)
                pbs_data[i][int(agent_num/2 - 1)].append(cost_stats[res][name][agent_num][scen])

                all_values.append(cost_stats[res][name][agent_num][scen])

    offset = 0.2

    # create three subplots - one for each resolution
    fig, axs = plt.subplots(3, 1, sharex=True, figsize=(8,5))
    
    max_value = 0
    for i in range(len(resolutions)): 
        for j, x in enumerate(x_data): 
            if len(pbs_data[i][j]) == 0: 
                continue
            true_x = np.linspace(x-offset, x+offset, len(cbs_data[i][j]))
            axs[i].scatter(true_x, cbs_data[i][j], color=cbs_color[i], marker="s", s=20)
            axs[i].scatter(true_x, pbs_data[i][j], color=pbs_color[i], marker="o", s=6)
            max_value = max(max_value, max(pbs_data[i][j]))
            
    # round maximum ratio value to the nearest thousandth, or set to 1.003 if all values are 1.0
    max_value = round(max_value, 3)
    if max_value == 1.0: 
        max_value = 1.003
    y_off = (max_value - 1.00) / 10
    
    for i in range(len(resolutions)): 
        # set y-axis label for each subplot
        axs[i].set_ylabel("Flowtime Ratio \nfor Res " + str(resolutions[i]), fontsize=9)
        
        # create dummy points for clean legends
        cbs_dummy = Line2D([0], [0], marker='s', color='w', markerfacecolor=cbs_color[i], markersize=7, label=cbs_label[i])
        pbs_dummy = Line2D([0], [0], marker='o', color='w', markerfacecolor=pbs_color[i], markersize=6, label=pbs_label[i])
        axs[i].legend(handles=[cbs_dummy, pbs_dummy], fontsize=8)

        # set y-axis limits and ticks
        axs[i].set_ylim(1 - y_off, max_value)
        axs[i].tick_params(axis='y', labelsize=8)  
            
    # Customize labels, titles, etc.
    # Customize labels, titles, legends, etc., as needed
    fig.suptitle('Suboptimality Comparison of ' + name + " Group",
                x=0.555, horizontalalignment='center')
    plt.xlabel('Agents', fontsize=9)
    plt.xticks(x_data, fontsize = 8)
    plt.tight_layout(rect=[0.05, 0.03, 1, 1])
    
    # Save and Show the plot
    plt.savefig("Figures/IndividualFigures/CostRatios_" + name + ".png")
    plt.show()
    